In [1]:
# import libraries
import pandas as pd
import networkx as nx

import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt

In [4]:
path = "/home/dhamarkin/Yo/Paper_Pablo/complexity/data_clean/"

In [6]:
# Read the data
df_EC = pd.read_csv(path + "PA_ruc.txt", delimiter='\t', low_memory=False) # PA RUC
ciiu = pd.read_csv(path+"ciiu4n4_english.csv", low_memory=False)

In [7]:
df_EC

,idanonimo,obligado_actual,codigo_clase_contrib,codigo_tipo_contribuyente_area,codigo_ciiu4n4_clase,codigo_canton,anio_cambio_obligado
0,1,N,OTR,1,L6820,21701,NaN
1,2,N,OTR,1,L6820,20101,NaN
2,3,N,OTR,1,A0141,20101,NaN
3,4,N,OTR,1,L6810,20101,NaN
4,5,N,OTR,1,G4610,21701,NaN
...,...,...,...,...,...,...,...
4270730,4268683,N,OTR,1,S9609,10901,NaN
4270731,4268684,N,OTR,1,V0300,21701,NaN
4270732,4268685,N,OTR,1,V0300,10804,NaN
4270733,4268686,N,OTR,1,V0300,21701,NaN


In [9]:
ciiu

,id,description
0,A,"Agriculture, forestry and fishing"
1,A01,"Crop and animal production, hunting and relate..."
2,A011,Growing of non-perennial crops
3,A0111,"Growing of cereals (except rice), leguminous c..."
4,A0112,Growing of rice
...,...,...
766,V,No economic activity - ISIC
767,W,As a dependent private sector
768,X,As a dependent public sector
769,Y,To Verify


In [10]:
# Rename some columns
ciiu = ciiu.rename(columns={'id': 'ciiu_4n4'})
df_EC.rename(columns={'idanonimo': 'id', 'codigo_tipo_contribuyente_area': 'contrib_area', 
                      'anio_cambio_obligado': 'anio', 'codigo_ciiu4n4_clase': 'ciiu_4n4',
                      'codigo_canton' :'city_code'}, inplace=True)

In [11]:
# Groupby df_EC con el id y el contrib_area
df_EC_ = df_EC.groupby(['id', 'contrib_area'])['anio'].sum().reset_index()

print ('Number of natural persons:')
print (df_EC_[(df_EC_['contrib_area']==1)].count())

print ('Number of legal persons:')
print (df_EC_[(df_EC_['contrib_area']>1)].count())

Number of natural persons:
id              3942095
contrib_area    3942095
anio            3942095
dtype: int64
Number of legal persons:
id              328640
contrib_area    328640
anio            328640
dtype: int64


In [12]:
df_EC

,id,obligado_actual,codigo_clase_contrib,contrib_area,ciiu_4n4,city_code,anio
0,1,N,OTR,1,L6820,21701,NaN
1,2,N,OTR,1,L6820,20101,NaN
2,3,N,OTR,1,A0141,20101,NaN
3,4,N,OTR,1,L6810,20101,NaN
4,5,N,OTR,1,G4610,21701,NaN
...,...,...,...,...,...,...,...
4270730,4268683,N,OTR,1,S9609,10901,NaN
4270731,4268684,N,OTR,1,V0300,21701,NaN
4270732,4268685,N,OTR,1,V0300,10804,NaN
4270733,4268686,N,OTR,1,V0300,21701,NaN


In [ ]:
df_EC = df_EC[['id','contrib_area', 'ciiu_4n4','city_code']] 

In [ ]:
df_EC['id'] = df_EC['id'].astype(int) 
df_EC['contrib_area'] = df_EC['contrib_area'].astype(int)

In [ ]:
years = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]

In [ ]:
# open the 3 df with diferent information
for year in years:
    y_int = str(year)
    df = pd.read_csv(path+'totaltax_noND('+y_int+').csv', low_memory=False) 
    df1 = df.groupby(['id_inf', 'id_prov'])['total_tax'].sum().reset_index()
    df1.loc[df1['total_tax'] > 2.20003e+09 ]  # no include outliers
    df1.loc[(df1['id_inf'] == '1801618') & (df1['id_prov'] == '2705583')]
    df1.loc[3083122, 'total_tax'] = 15000
    df1.loc[(df1['id_inf'] == '1801618') & (df1['id_prov'] == '2705583')]
    df2 = pd.merge(df1, df_EC, how='left', left_on=['id_prov'], right_on=['id'])
    df3 = pd.merge(df2, df_EC, how='left', left_on=['id_inf'], right_on=['id'], suffixes=('_pr', '_inf'))
    df4 = df3.groupby(['ciiu_4n4_pr', 'ciiu_4n4_inf'])['total_tax'].sum().reset_index()
    df4['ciiu_2n2_pr'] = df4['ciiu_4n4_pr'].str[:3]
    df4['ciiu_2n2_inf'] = df4['ciiu_4n4_inf'].str[:3]
    df5 = df4.groupby(['ciiu_2n2_pr', 'ciiu_2n2_inf'])['total_tax'].sum().reset_index()
    df4 = df4[df4.ciiu_4n4_pr != "99999"]
    df4 = df4[df4.ciiu_4n4_inf != "99999"]
    G = nx.Graph()
    G.add_nodes_from(df4['ciiu_4n4_pr'])
    G.add_nodes_from(df4['ciiu_4n4_inf'])
    edges = [(row['ciiu_4n4_pr'], row['ciiu_4n4_inf']) for index, row in df4.iterrows()]
    G.add_edges_from(edges)
    avg_clustering = nx.average_clustering(G)
    degree_centrality = nx.degree_centrality(G)
    nx.set_node_attributes(G, degree_centrality, 'degree_centrality')
    weighted_degree_centrality = nx.degree_centrality(G)
    nx.set_node_attributes(G, weighted_degree_centrality, 'weighted_degree_centrality')
    closeness_centrality = nx.closeness_centrality(G)
    nx.set_node_attributes(G, closeness_centrality, 'closeness_centrality')
    betweenness_centrality = nx.betweenness_centrality(G)
    nx.set_node_attributes(G, betweenness_centrality, 'betweenness_centrality')
    eigenvector_centrality = nx.eigenvector_centrality(G)
    nx.set_node_attributes(G, eigenvector_centrality, 'eigenvector_centrality')
    clustering = nx.clustering(G)
    nx.set_node_attributes(G, clustering, 'clustering')
    pagerank = nx.pagerank(G)
    nx.set_node_attributes(G, pagerank, 'pagerank')
    df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
    df_nodes.reset_index(inplace=True)
    df_nodes.rename(columns={'index': 'ciiu_4n4'}, inplace=True)
    df_nodes = pd.merge(df_nodes, ciiu, how='left', on='ciiu_4n4')
    df_nodes['year'] = year
    df_nodes.to_csv(path+'df_nodes_'+y_int+'.csv', index=False)
    
